In [1]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table

In [2]:
engine = create_engine('sqlite:///batting.sql', echo=False)

In [14]:
engine.table_names()

['batting', 'bowling']

In [4]:
# Read the score card
df = pd.read_csv("../data/scores.csv")

In [5]:
# Renaming the Unnamed column
df.rename(columns={'Unnamed: 1':'FOW'}, inplace=True)
# Add innings column to the dataframe
df['Innings'] = 1 # Default 1

In [6]:
# Row indexes where the bowling and batting scorecard for different innings begin
bowler_indexes = df.loc[df['BATSMEN'] == 'Bowling'].index.tolist()
batsmen_indexes = df.loc[df['BATSMEN'] == 'BATSMEN'].index.tolist()
# First batting index as -1 because first innings starts at index 0
batsmen_indexes = [-1] + batsmen_indexes

In [7]:
# Break score card into innings
innings = [] # Row indexes of where the inning starts, where the bowling starts and where inning ends
for counter, ini in enumerate(batsmen_indexes):
    # Last inning ends at the end of the data frame so df.shape[0]
    if counter == len(batsmen_indexes) - 1:
        innings.append((batsmen_indexes[counter] + 1, bowler_indexes[counter] + 1, df.shape[0]))
    else:
        innings.append((batsmen_indexes[counter] + 1, bowler_indexes[counter] + 1, batsmen_indexes[counter + 1]))

In [8]:
# Mapping to change the column names for bowling
bat_to_bowl = {'BATSMEN':'BOWLER', 'FOW':'O', 'R':'M', 'B':'R', 'M':'W', '4s':'Econ', '6s':'WD', 'SR':'NB'}
Batting = []
Bowling = []
# Change type of columns to make querying in sql possible
column_to_numeric_bat = ["R", "B", "M", "4s", "6s", "SR", 'Innings']
column_to_numeric_bowl = ["O", "M", "R", "W", "Econ", "WD", "NB", 'Innings']
# Make list of separate dataframes for bowling and batting of each innings
for counter, inning in enumerate(innings):
    # Replace deafult innings value by the real value
    df.loc[innings[counter][0]:innings[counter][1] - 1, 'Innings'].replace(1, counter + 1, inplace=True)
    # For batting
    bat = df[innings[counter][0]:innings[counter][1] - 1]
#     bat['Innings'].replace(1, counter + 1, inplace=True)
    bat[column_to_numeric_bat] =  bat[column_to_numeric_bat].apply(pd.to_numeric)
    Batting.append(bat)
    # For Bowling
    bowl = df[innings[counter][1]:innings[counter][2]]
#     bowl['Innings'].replace(1, counter + 1, inplace=True)    
    column_changed_bowl = bowl.rename(columns = bat_to_bowl)
    column_changed_bowl[column_to_numeric_bowl] =  column_changed_bowl[column_to_numeric_bowl].apply(pd.to_numeric)    
    Bowling.append(column_changed_bowl)

/home/samyak/.local/lib/python3.6/site-packages/pandas/core/frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [9]:
Batting[0]

,BATSMEN,FOW,R,B,M,4s,6s,SR,Innings
0,PP Shaw,c & b Bishoo,134,154,238,19,0,87.01,1
1,KL Rahul,lbw b Gabriel,0,4,6,0,0,0.00,1
2,CA Pujara,c †Dowrich b Lewis,86,130,194,14,0,66.15,1
3,V Kohli (c),c Bishoo b Lewis,139,230,341,10,0,60.43,1
4,AM Rahane,c †Dowrich b Chase,41,92,127,5,0,44.56,1
5,RR Pant †,c Paul b Bishoo,92,84,121,8,4,109.52,1
6,RA Jadeja,not out,100,132,152,5,5,75.75,1
7,R Ashwin,c †Dowrich b Bishoo,7,15,18,1,0,46.66,1
8,Kuldeep Yadav,lbw b Bishoo,12,32,33,2,0,37.50,1
9,UT Yadav,c Lewis b Brathwaite,22,24,22,0,2,91.66,1


In [10]:
Bowling[0]

,BOWLER,O,M,R,W,Econ,WD,NB,Innings
12,ST Gabriel,21.0,1,84,1,4.00,0,2,1
13,KMA Paul,15.0,1,61,0,4.06,0,2,1
14,SH Lewis,20.0,0,93,2,4.65,0,0,1
15,D Bishoo,54.0,3,217,4,4.01,0,0,1
16,RL Chase,26.0,1,137,1,5.26,0,0,1
17,KC Brathwaite,13.5,1,47,1,3.39,0,0,1


In [11]:
# Check if the table exists, if not then populate it
if engine.dialect.has_table(engine.connect(), "batting"):
    pass
else:
    for x in range(len(Batting)):
        Batting[x].to_sql(name='batting', con=engine, if_exists='append')
        Bowling[x].to_sql(name='bowling', con=engine, if_exists='append')

In [13]:
engine.execute("SELECT BATSMEN FROM batting where R > 100").fetchall()

[('PP Shaw',), ('V Kohli (c)',)]